#                      -- DESASTER --
Simulating household reconstruction with Discrete Event Simulation 

Requires python 3.4

In [854]:
#stdlib and 3rd Party imports
import sys, datetime, os
import numpy as np
import pandas as pd
import simpy
from simpy.util import start_delayed
import folium
from folium.plugins import MarkerCluster
import seaborn
%matplotlib inline

#add path to desaster module, later we'll install this into site-packages so we shouldn't need to do this
sys.path.append("/Users/geomando/Dropbox/github/SeaGrantSimulation")
#import desaster files
from desaster import entities, capitals, request, io, movement, search, rebuild


Here we're importing the modules and setting up the stuff

## Load input files for the scenario

In [855]:
# #scenario_file = '../inputs/scenario_test1.xlsx'
# scenario_file = "../inputs/household_inputs.xlsx"

# # Create Pandas dataframe of attribute data for all households to be modeled in the simulation
# # required column names, exactly as written: Name , Savings , Insurance
# households_df = pd.read_excel(scenario_file, sheetname='households')

# # Create Pandas dataframe of attribute data for all vacant homes (housing stock) to be modeled in the simulation
# housing_stock_df = pd.read_excel(scenario_file, sheetname='housing_stock')

# # Set input data for all human capital types, as dict or Pandas Series
# # .loc stuff is to convert the DataFrame to a Series ... data will function the same as a dictionary as well
# human_cap_data = pd.read_excel(scenario_file, sheetname='human_capital', index_col=0).iloc[:,0]

# # Set input data for all financial capital types, as dict or Pandas Series
# financial_cap_data = pd.read_excel(scenario_file, sheetname='financial_capital', index_col=0).iloc[:,0]

### Randomize households and reset index

In [856]:
# households_df = households_df.sample(frac=1).reset_index(drop=True)
# #takes a random sample, frac is a fraction to sample (1 means take a 100% sample), 
# #reset index drops the old scrambled index and puts in a fresh ascending count
# households_df.head()

# Initiate Simulation

In [857]:
simulation = simpy.Environment() 

In [858]:
write_story = False #do we want the story of each household?

## Scenario Definitions

In [859]:
fraction = 1.0
money_patience = 180  # days until give up the search for rebuild money
home_patience = money_patience  # days until give up the search for a new home
scenario_name = '{0}PctRes_MonPat{1}_YesHomeSearch'.format(int(fraction*100),int(home_patience))
output_path = "../Outputs/out_{0}.csv".format(scenario_name)

In [860]:
human_cap_data["Contractors"] = int(len(households_df) * fraction)
human_cap_data["Engineers"] = int(len(households_df) * fraction)
human_cap_data["FEMA Processors"]= int(len(households_df) * fraction)
human_cap_data["Inspectors"] = int(len(households_df) * fraction)
human_cap_data["Insurance Adjusters"] = int(len(households_df) * fraction)
human_cap_data["Loan Processors"] = int(len(households_df) * fraction)
human_cap_data["Permit Processors"] = int(len(households_df) * fraction)

total_damage = 29533906.0 # Sum of all damage values in PC
financial_cap_data['Building Materials'] = total_damage  * fraction
financial_cap_data['FEMA Aid'] = total_damage * fraction

In [861]:
# # Infinite Resources

# scenario_name = 'InfRes_MonPat365_YesHomeSearch'
# output_path = "../Outputs/out_{0}.csv".format(scenario_name)

# human_cap_data["Contractors"] = 100000000000000
# human_cap_data["Engineers"] = 100000000000000
# human_cap_data["FEMA Processors"]= 100000000000000
# human_cap_data["Inspectors"] = 100000000000000
# human_cap_data["Insurance Adjusters"] = 100000000000000
# human_cap_data["Loan Processors"] = 100000000000000
# human_cap_data["Permit Processors"] = 100000000000000

# total_damage = 29533906.0 # Sum of all damage values in PC
# financial_cap_data['Building Materials'] = total_damage
# financial_cap_data['FEMA Aid'] = total_damage

In [862]:
financial_capital = capitals.FinancialCapital(simulation, financial_cap_data) #resource

human_capital = capitals.HumanCapital(simulation, human_cap_data) #resource

households = entities.importHouseholds(simulation, households_df, write_story) #entity object container

housing_stock = capitals.importHousingStock(simulation, housing_stock_df) #available housing

In [873]:
len(households)

2860

### Write a function that controls the flow for each household.

In [863]:
# def master_process(simulation, human_capital, financial_capital, entity, write_story):
        
#     yield simulation.process(request.inspection(simulation, human_capital, entity.residence, entity, write_story))
    
#     # Specify the event sequence for households from the time of the hazard through the decisions to relocate 
#     # or rebuild
#     if entity.residence.damage_state != 'None':
        
#         money_patience = 1000  # days until give up the search for rebuild money

#         # Search for rebuild money
#         yield simulation.process(search.rebuild_money(simulation, human_capital, 
#                                                         financial_capital, entity, 
#                                                         money_patience, write_story))
        
#         if entity.gave_up_money_search == True:
#                 return
        
#         # If home is completely damaged, search for a new home to purchase.
#         if entity.residence.damage_state == 'Complete':
            
#             home_patience = 550  # days until give up the search for a new home

#             search_outcome = yield simulation.process(search.permanent_housing(simulation, entity, home_patience, housing_stock, human_capital, write_story))

#             if entity.gave_up_home_search == True:
#                 return

#         if entity.residence.damage_state != 'None':
#             yield simulation.process(request.engineering_assessment(simulation, human_capital, entity, write_story))

#             yield simulation.process(request.permit(simulation, human_capital, entity, write_story))

#             yield simulation.process(rebuild.home(simulation, human_capital, financial_capital, entity, write_story))

In [864]:
def master_process(simulation, human_capital, financial_capital, entity, money_patience, home_patience, write_story):
        
    yield simulation.process(request.inspection(simulation, human_capital, entity.residence, entity, write_story))
    
    # Specify the event sequence for households from the time of the hazard through the decisions to relocate 
    # or rebuild
    if entity.residence.damage_state != 'None':
        


        # Search for rebuild money
        yield simulation.process(search.rebuild_money(simulation, human_capital, 
                                                        financial_capital, entity, 
                                                        money_patience, write_story))
        
        if entity.gave_up_money_search == True:
            

            search_outcome = yield simulation.process(search.permanent_housing(simulation, entity, home_patience, housing_stock, human_capital, write_story))

            if entity.gave_up_home_search == True:
                return

        if entity.residence.damage_state != 'None':
            yield simulation.process(request.engineering_assessment(simulation, human_capital, entity, write_story))

            yield simulation.process(request.permit(simulation, human_capital, entity, write_story))

            yield simulation.process(rebuild.home(simulation, human_capital, financial_capital, entity, write_story))

In [865]:
# Initiate a master process for each household to be modeled in the simulation
for i in range(len(households)):
    simulation.process(master_process(simulation, human_capital, financial_capital, households[i], 
                                      money_patience, home_patience, write_story))

In [866]:
undamaged_housing = 0
for i in housing_stock.items:
    if i.damage_state == "None":
        undamaged_housing += 1
        
print (undamaged_housing)

16


## Rebuild the housing stock

In [867]:
# # Do inspections on all of the vacant homes in the housing stock
# for home in housing_stock.items:
#     simulation.process(request.inspection(simulation, human_capital, home))

# # Schedule an event that randomly fixes moderately or completely damaged homes in the vacant housing stock
# # with probability = fix_probability
# fix_probability = 1.0
# fix_schedule = 100

# start_delayed(simulation, rebuild.stock(simulation, housing_stock, fix_probability), fix_schedule)

In [868]:
#Reload building material at a preordained time
# start_delayed(simulation, capitals.reloadBuildingMaterial(simulation, financial_capital.building_materials, amount = 100000000), 100)

## Run the model

In [869]:
simulation.run()

## Outputs to verify model ran correctly

In [870]:
# num_undamaged = 0
# num_rebuilt = 0
# num_gave_up_money_search = 0
# num_relocated = 0
# num_gave_up_home_search = 0
# num_moved = 0
# num_loaned = 0
# for household in households:
#     if household.residence.damage_state_start == 'None': num_undamaged += 1
#     if household.home_get is not None: num_rebuilt += 1
#     if household.gave_up_money_search: num_gave_up_money_search += 1   
#     if household.home_search_stop is not None:
#         if household.home_search_stop > 0.0: num_relocated += 1
#     if household.gave_up_home_search: num_gave_up_home_search += 1
#     if household.loan_get: num_loaned += 1    
# print('{0} out of {1} households suffered no damage to their homes.\n'.format(num_undamaged, len(households)),
#       '{0} out of {1} households rebuilt or repaired their damaged home.\n'.format(num_rebuilt, len(households)),
#         '{0} out of {1} households gave up searching for money.\n'.format(num_gave_up_money_search, len(households)),
#         '{0} out of {1} households decided to find a new home.\n'.format(num_relocated, len(households)),
#         '{0} out of {1} households gave up searching for a home. \n'.format(num_gave_up_home_search, len(households)),
#       '{0} out of {1} households applied for and received a loan.'.format(num_loaned, len(households))
#      )

# MAKE A NEW DATAFRAME FOR EXPORT

In [871]:
#fills the empty dataframe we made above for the output. incredibly badly written
a = [
 'inspection_get',
 'assistance_get',
 'assessment_get',
 'permit_get',
    'claim_get',
    'loan_get',
 'home_get',
    'gave_up_money_search',
    'gave_up_home_search',
    'story']
a.append("latitude");a.append("longitude") #add stuff we do want
a.append("damage_state_start");a.append("damage_state") #add stuff we do want
a.append("damage_value_start");a.append("damage_value") #add stuff we do want
a = sorted(a)
df = pd.DataFrame(columns=a, dtype=int)
iters = 0
att_itter = 0
new_column={}
log = []
for i in households: #loop through all entities
    i.story = i.story_to_text()
    i.latitude = i.residence.latitude #extracting lat and long from the residence object
    i.longitude = i.residence.longitude
    i.gave_up_money_search = int(i.gave_up_money_search)
    i.gave_up_home_search = int(i.gave_up_home_search)
    i.damage_state_start = i.residence.damage_state_start
    i.damage_state = i.residence.damage_state
    i.damage_value = i.residence.damage_value
    i.damage_value_start = i.residence.damage_value_start
    for att in a: #loop through the attributes in our list of column names we want
        try:
            new_column[att] = i.__getattribute__(att) #set the b dictionary
            #mydata[att]= i.__getattribute__(att)
            
        except ValueError:
            new_column[att] = np.nan
        except AttributeError as e:
            new_column[att] = np.nan
            log.append("Household {0} had an attr error, {1}".format(i.name, e))
        finally:
            att_itter += 1
    mydata=pd.DataFrame([new_column]) #this turns our newly made column into a database where it can be combined with the df

    df = df.append(mydata, ignore_index=True)

    iters += 1
    

# output_path = "../outputs/Output{0}.csv".format(datetime.datetime.now().strftime("%m%d%y%I%M"))

df.to_csv(output_path)

In [872]:
df.columns

Index(['assessment_get', 'assistance_get', 'claim_get', 'damage_state',
       'damage_state_start', 'damage_value', 'damage_value_start',
       'gave_up_home_search', 'gave_up_money_search', 'home_get',
       'inspection_get', 'latitude', 'loan_get', 'longitude', 'permit_get',
       'story'],
      dtype='object')